In [2]:
import torch
import numpy as np
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time


# Inicializa navegador
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://contexto.me/en/")

print("🔄 Carregando BERT...")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")
model.eval()

# Embeddings cache
embeddings = {}
memory = {}


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\Albert\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\Albert\anaconda3\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\Albert\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start
    self.io_loop.start()
  File "c:\Users\Albert\anaconda3\Lib\site-

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.




A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\Albert\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\Albert\anaconda3\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\Albert\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start
    self.io_loop.start()
  File "c:\Users\Albert\anaconda3\Lib\site-

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



🔄 Carregando BERT...


In [3]:
# Palavras genéricas para abrir sentidos semânticos
temas_exploratorios = [
    "object", "place", "person", "animal", "emotion",
    "food", "vehicle", "technology", "family", "music",
    "clothing", "tool", "plant", "profession", "color"
]
temas_usados = set()

# Função para obter embedding
def get_embedding(word):
    if word in embeddings:
        return embeddings[word]

    with torch.no_grad():
        inputs = tokenizer(word, return_tensors="pt", truncation=True)
        inputs = {k: v for k, v in inputs.items()}
        outputs = model(**inputs)
        emb = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
        embeddings[word] = emb
        return emb

# Adiciona chute manual
def adicionar_chute(palavra, score):
    print(f"📥 Chute registrado: '{palavra}' com score {score}")
    memory[palavra] = score
    get_embedding(palavra)

# Sugestão adaptativa
def sugerir_proximo():
    print("\n🎯 Gerando nova sugestão...")

    if not memory:
        sugestao = random.choice(temas_exploratorios)
        temas_usados.add(sugestao)
        print(f"🧭 Começando exploração: {sugestao}")
        return sugestao

    # Classificação refinada por score
    perfeitos = [(p, s) for p, s in memory.items() if s < 50]
    otimos    = [(p, s) for p, s in memory.items() if 50 <= s < 200]
    bons      = [(p, s) for p, s in memory.items() if 200 <= s < 300]
    oks       = [(p, s) for p, s in memory.items() if 300 <= s < 500]
    ruins     = [(p, s) for p, s in memory.items() if s >= 500]

    todos_ruins = len(ruins) == len(memory)

    if todos_ruins:
        restantes = [t for t in temas_exploratorios if t not in temas_usados]
        if restantes:
            sugestao = random.choice(restantes)
            temas_usados.add(sugestao)
            print(f"🔍 Todos os chutes estão ruins. Explorando novo tema: {sugestao}")
            return sugestao
        else:
            print("⚠️ Todos os temas exploratórios já foram usados. Chutando palavra genérica aleatória.")
            return random.choice([w for w in tokenizer.vocab.keys() if w.isalpha() and len(w) > 4 and w not in memory])

    # Refinamento inteligente com prioridade por nível
    base = []

    if len(perfeitos) >= 1:
        base = sorted(perfeitos, key=lambda x: x[1])  # usa todos
        print(f"💥 Refinando a partir de perfeitos ({len(base)}): {[p for p, _ in base]}")
    elif len(otimos) >= 1:
        base = sorted(otimos, key=lambda x: x[1])
        print(f"🌟 Refinando a partir de ótimos ({len(base)}): {[p for p, _ in base]}")

    elif len(bons) >= 1:
        base = sorted(bons, key=lambda x: x[1])
        print(f"👌 Refinando a partir de bons ({len(base)}): {[p for p, _ in base]}")

    elif len(oks) >= 1:
        base = sorted(oks, key=lambda x: x[1])
        print(f"🙂 Refinando a partir de oks ({len(base)}): {[p for p, _ in base]}")
    else:
        base = sorted(memory.items(), key=lambda x: x[1])[:2]
        print(f"🤔 Refinando com os menos ruins: {[p for p, _ in base]}")

    # Cálculo do vetor médio com pesos
    vetores = np.array([get_embedding(p) for p, _ in base])
    pesos = np.array([1 / (s + 1) for _, s in base])
    pesos = pesos / pesos.sum()
    vetor_medio = np.average(vetores, axis=0, weights=pesos)

    # Seleção de candidatos
    candidatos = [w for w in tokenizer.vocab.keys() if w.isalpha() and len(w) > 4 and w not in memory]
    random.shuffle(candidatos)
    candidatos = candidatos[:1000]

    melhor_palavra = None
    melhor_sim = -1
    for w in candidatos:
        vec = get_embedding(w)
        sim = cosine_similarity([vetor_medio], [vec])[0][0]
        if sim > melhor_sim:
            melhor_sim = sim
            melhor_palavra = w

    print(f"✅ Próxima sugestão: {melhor_palavra} (sim={melhor_sim:.4f})")
    return melhor_palavra

def enviar_chute_site(palavra):
    # Preenche o campo
    input_box = driver.find_element(By.CSS_SELECTOR, "input.word")
    input_box.clear()
    input_box.send_keys(palavra)
    input_box.send_keys(Keys.ENTER)

    # Aguarda resposta
    time.sleep(1.5)

    # Verifica se apareceu a mensagem de palavra repetida
    try:
        msg_box = driver.find_element(By.CSS_SELECTOR, ".message-text")
        if "already guessed" in msg_box.text:
            print(f"⚠️ Palavra já testada: {palavra}")
            return palavra, None  # Ignorar
    except:
        pass  # nenhuma mensagem = segue normal

    # Lê o chute mais recente
    try:
        resultado = driver.find_element(By.CSS_SELECTOR, ".row-wrapper.current .row")
        texto = resultado.text.strip()
        palavra_retornada, score = texto.split()
        return palavra_retornada.lower(), int(score)
    except Exception as e:
        print("⚠️ Erro ao tentar ler o resultado:", e)
        return palavra, None

def reenviar_chutes_para_interface():
    print("\n📤 Reenviando chutes manuais para a interface do jogo...")
    for palavra in list(memory.keys()):
        _, score_checado = enviar_chute_site(palavra)  # reenviar no site e confirmar score real
        print(f"🔁 {palavra} → confirmado no site com score {score_checado}")

def adicionar_chute_site(palavra):
    print(f"\n📤 Enviando chute direto: '{palavra}'")
    chute, score = enviar_chute_site(palavra)

    if score is None:
        print(f"⏭️ Ignorado: '{chute}' não retornou score (talvez repetido ou erro).")
        return

    memory[chute] = score
    get_embedding(chute)
    print(f"📥 Registrado: '{chute}' com score {score}")


In [4]:
time.sleep(10)
def loop_automatico():
    reenviar_chutes_para_interface()

    while True:
        proxima = sugerir_proximo()
        chute, score = enviar_chute_site(proxima)
        
        if score is not None:
            adicionar_chute(chute, score)
        else:
            print("⏭️ Ignorando chute inválido ou repetido.")

        if score == 0:
            print("🎉 Palavra secreta encontrada:", chute)
            break


In [ ]:
listaPalavras = []

if len(listaPalavras) != 0:
    print("⚠️ Nenhuma palavra foi passada para o loop automático.") 
    for palavra in listaPalavras:
        adicionar_chute_site(palavra)
else:
    print("🔄 Iniciando jogo")
loop_automatico()



📤 Enviando chute direto: 'plant'
📥 Registrado: 'plant' com score 349

📤 Reenviando chutes manuais para a interface do jogo...
⚠️ Palavra já testada: plant
🔁 plant → confirmado no site com score None

🎯 Gerando nova sugestão...
🙂 Refinando a partir de oks (1): ['plant']


KeyboardInterrupt: 